# Calculate $\omega \text{s}$ and corresponding eigenvectors

In [7]:
from meclat import MechanicalLattice1D

In [8]:
import numpy as np

ml1d = MechanicalLattice1D(k=[.7, 1.3], m=[1.3, .7], precision=0.1)
ws, evs = ml1d.dispersion()

# Animation

In [9]:
import numpy as np
import plotly.graph_objects as go

# Parameters
q = np.pi
N = 30
idx = min(range(len(ml1d.qs)), key=lambda i: abs(ml1d.qs[i]-q))
mode = 0 # 0 for phonon, 1 for photon
w = ws[idx, mode] # /s

# Print infos
print("Angular frequency: ", w)
print("Wave number: ", q)

# Parameters for animation
fps = 10 # /s
animation_length = 3 # s

frames = []
for t in range(animation_length * fps):
    dt = t / fps
    dphase = dt * w * np.pi
    y = []
    for i in range(N):
        y.append(evs[idx, mode, 0] * np.exp(1.j * (q * i + dphase)))
        y.append(evs[idx, mode, 1] * np.exp(1.j * (q * i + dphase)))
    y = np.array(y)
    frames.append(go.Frame(data=[go.Scatter(y=y.real, line_shape='spline')]))

fig = go.Figure(
    data=frames[0].data,
    layout=go.Layout(
        title="Dispersion relation animation",
        yaxis=dict(range=[-1, 1], autorange=False),
        updatemenus=[dict(
            type="buttons",
            buttons=[dict(label="Play",
                            method="animate",
                            args=[None, {"frame": {"duration": 1000 / fps, 
                                                           "redraw": False},
                                                 "fromcurrent": True, 
                                                 "transition": {"duration": 100}}]),
                     dict(label="Pause",
                            method="animate",
                            args=[[None], {"frame": {"duration": 0, "redraw": False},
                                  "mode": "immediate",
                                  "transition": {"duration": 0}}],
                     )
                    ])]
    ),
    frames=frames[1:])
fig.show()

$\omega$ Angular frequency:  1.1326024762262508
Wave number:  3.141592653589793


# Dispersion relation

In [10]:
import pandas as pd
import numpy as np

w0 = ws[:, 0]
w1 = ws[:, 1]
ws = np.append(w0, w1)

x = np.append(ml1d.qs, ml1d.qs)
y = ws
index = np.append(np.repeat(0, len(ml1d.qs)), np.repeat(1, len(ml1d.qs)))


df = pd.DataFrame({
    "q": x,
    "w": y,
    "index": index,
})

In [11]:
import plotly.express as px

fig = px.line(df, x="q", y="w", color='index')
fig.show()